<b>Nathan Gulley<br>
MSDS 600<br>
Week 5 Assignment - Data Science Automation</b><br>

#### 1. Load and inspect the raw churn data

In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore") 

In [6]:
df = pd.read_csv('churn_data_raw.csv', index_col='customerID')
df.head()

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,
7590-VHVEG,1,No,Month-to-month,Electronic check,29.85,29.85,No
5575-GNVDE,34,Yes,One year,Mailed check,56.95,1889.50,No
3668-QPYBK,2,Yes,Month-to-month,Mailed check,53.85,108.15,Yes
7795-CFOCW,45,No,One year,Bank transfer (automatic),42.30,1840.75,No
9237-HQITU,2,Yes,Month-to-month,Electronic check,70.70,151.65,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tenure          7043 non-null   int64  
 1   PhoneService    7043 non-null   object 
 2   Contract        7043 non-null   object 
 3   PaymentMethod   7043 non-null   object 
 4   MonthlyCharges  7043 non-null   float64
 5   TotalCharges    7032 non-null   float64
 6   Churn           7043 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 440.2+ KB


#### 2. Use pycaret to determine the best performing ML algorithm on the churn data

In [7]:
#!pip install --user pycaret

from pycaret.classification import ClassificationExperiment

For supervised learning, Pycaret supports two types of experiment classes. ClassificationExperiement and RegressionExperiment. Since our target is categorical, we will create an instance of the ClassificationExperiemnt

In [ ]:
automl = ClassificationExperiment()

Calling the <b>setup()</b> function on the ClassificationExperiement instance allows us to automate many of the steps that are required to turn raw data into the prepped data required by ML algorithms: 

- handling of missing values 
- handling of outlier values
- conversion of categories to numbers
- feature engineering
- feature selection

It also automates the process of splitting prepped data into training and test sets

In [8]:
automl.setup(df, target='Churn')

,Description,Value
0,Session id,6129
1,Target,Churn
2,Target type,Binary
3,Target mapping,"No: 0, Yes: 1"
4,Original data shape,"(7043, 7)"
5,Transformed data shape,"(7043, 12)"
6,Transformed train set shape,"(4930, 12)"
7,Transformed test set shape,"(2113, 12)"
8,Numeric features,3
9,Categorical features,3


Calling the <b>compare_models()</b> function on the ClassificationExperiment automates that training of a diverse set of machine learning models on the training data and returns the best performing model in terms of accuracy. It provides a broad range of metrics that we can use to make an informed judgment about the performance of each model.

In [9]:
best_model = automl.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7929,0.8332,0.7929,0.7817,0.7834,0.4264,0.4332,0.1880
lr,Logistic Regression,0.7917,0.8307,0.7917,0.7815,0.7838,0.4296,0.4345,0.9080
ada,Ada Boost Classifier,0.7909,0.8343,0.7909,0.7794,0.7813,0.4211,0.4275,0.1450
lda,Linear Discriminant Analysis,0.7890,0.8212,0.7890,0.7804,0.7829,0.4301,0.4334,0.0450
ridge,Ridge Classifier,0.7872,0.8212,0.7872,0.7734,0.7741,0.3973,0.4078,0.0530
lightgbm,Light Gradient Boosting Machine,0.7860,0.8228,0.7860,0.7751,0.7779,0.4140,0.4185,0.2090
rf,Random Forest Classifier,0.7797,0.8051,0.7797,0.7700,0.7728,0.4033,0.4067,0.2370
et,Extra Trees Classifier,0.7671,0.7836,0.7671,0.7596,0.7618,0.3783,0.3808,0.1850
svm,SVM - Linear Kernel,0.7663,0.7243,0.7663,0.7475,0.7368,0.2910,0.3181,0.0580
knn,K Neighbors Classifier,0.7653,0.7438,0.7653,0.7504,0.7540,0.3475,0.3536,0.0720


In [10]:
best_model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=6129, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

Calling the <b>evaluate_model()</b> function on the ClassificationExperiment performs a deeper analysis of the given model from a variety of perspectives

In [11]:
automl.evaluate_model(best_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

Once we have chosen the best model from the mix, we can use it to make churn predictions from our original set of data. A variety of metrics allow us to evaluate the performance of the model as well as do a row by row comparison of the model's predicted value with the actual value of the target.

In [13]:
automl.predict_model(best_model, df.iloc[1:10])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8889,1.0000,0.8889,0.9074,0.8860,0.7692,0.7906


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,prediction_label,prediction_score
customerID,,,,,,,,,
5575-GNVDE,34,Yes,One year,Mailed check,56.950001,1889.500000,No,No,0.9308
3668-QPYBK,2,Yes,Month-to-month,Mailed check,53.849998,108.150002,Yes,No,0.5642
7795-CFOCW,45,No,One year,Bank transfer (automatic),42.299999,1840.750000,No,No,0.8655
9237-HQITU,2,Yes,Month-to-month,Electronic check,70.699997,151.649994,Yes,Yes,0.6376
9305-CDSKC,8,Yes,Month-to-month,Electronic check,99.650002,820.500000,Yes,Yes,0.7747
1452-KIOVK,22,Yes,Month-to-month,Credit card (automatic),89.099998,1949.400024,No,No,0.7138
6713-OKOMC,10,No,Month-to-month,Mailed check,29.750000,301.899994,No,No,0.8953
7892-POOKP,28,Yes,Month-to-month,Electronic check,104.800003,3046.050049,Yes,Yes,0.5336
6388-TABGU,62,Yes,One year,Bank transfer (automatic),56.150002,3487.949951,No,No,0.9389


#### 3. Save the model to disk

Pycaret allows us to save our model to disk. This makes it easier to reference the model in Python scripts and deploy it into production environments.

In [14]:
automl.save_model(best_model, 'pycaret_churn_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'MonthlyCharges',
                                              'TotalCharges'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,...
                                             criterion='friedman_mse', init=None,
                                             learning_rate=0.1, loss='log_loss',
                                             max_

#### 4. Create a Python script with a function that predicts the probability of churn for each row in the new_churn_data.csv file

Our <b>predict_churn_pycaret.py</b> Python script first calls the <b>load_model()</b> function to load the model that we have saved to disk into memory. It then calls the <b>predict_model()</b> function to use the specified model in making a prediction for each row in the specified dataframe.

In [18]:
from IPython.display import Code

Code('predict_churn_pycaret.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads churn data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('pycaret_churn_model')
    predictions = predict_model(model, data=df)
    predictions.rename({'Label': 'prediction_label'}, axis=1, inplace=True)
    return predictions['prediction_label']


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    #df = load_data('churn_data_raw.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions.head())

We use the magic command to run our Python script inside of Jupyter Notebooks:

In [19]:
%run predict_churn_pycaret.py

Transformation Pipeline and Model Successfully Loaded
predictions:
customerID
9305-CKSKC    No
1452-KNGVK    No
6723-OKKJM    No
7832-POPKP    No
6348-TACGU    No
Name: prediction_label, dtype: object


#### 5. Summary

In this notebook we explored applying a data science automation tool called Pycaret to our Telecommunications company's customer churn dataset. We demonstrated how a tool like Pycaret allows us to automate tasks that we previously did manually. 

1. Turning raw data into prepped data 
2. Splitting prepped data into training and test sets
3. Training a diverse set of machine learning models on the training data
4. Tuning ML model hyperparameters to achieve the optimum results
5. Evaluating and comparing the performance of each ML model to determine which one is best for solving the problem at hand

We also explored how to deploy our model by using Pycaret to save it to disk then using a Plython script to load and execute our model against a set of churn test data.